In [ ]:
# import the urllib library
import urllib.request
from datetime import date
import numpy as np
import matplotlib.pyplot as plt 
import json
from GHCND import *

In [ ]:
ghn = GHNCD()
ghn.readCountriesFile()
ghn.readStationsFile()

In [ ]:

# Get list of station names
station_names = ghn.getStatKeyNames()

# get url for a given station index
whichStat1 = 9
fileName = station_names[whichStat1]+'.dly'
urlName = 'http://www.hep.ucl.ac.uk/undergrad/0056/other/projects/ghcnd/ghcnd_gsn/' + fileName

# copy station data from remote to local
destination = f"data/{fileName}"
urllib.request.urlretrieve(urlName, destination)
statDict1 = ghn.processFile(destination)
print(ghn.getStation(station_names[whichStat1]))


In [ ]:
var1 = Variable(ghn.getVar(statDict1, 'TMAX'), "max temp (degC)", ghn.stationDict[station_names[whichStat1]].name)
var1.convert_time()

fig, ax = plt.subplots()
ax.plot(var1.get_dates(), var1.get_vals())
ax.set_xlabel("Days since first recording")
ax.set_ylabel(f"{var1.get_label()}")
ax.legend((var1.get_station()))


### Find station with the least number of gaps in max_temp data

In [ ]:
# get list of station names
station_names = ghn.getStatKeyNames()

# count number of gaps in each station
counts = {}
for station in station_names:
    try:
        # get url for a given station index
        fileName = f"{station}.dly"
        urlName = 'http://www.hep.ucl.ac.uk/undergrad/0056/other/projects/ghcnd/ghcnd_gsn/' + fileName

        # copy station data from remote to local
        destination = f"data/{fileName}"
        urllib.request.urlretrieve(urlName, destination)
        statDict = ghn.processFile(destination)
        print(ghn.getStation(station))

        var = Variable(ghn.getVar(statDict, 'TMAX'), "max temp (degC)", ghn.stationDict[station].name)
        var.convert_time()
        count = var.count_gaps()
        counts[station] = count
    except Exception:
        print(f"Gaps couldn't be counted for station {station}")
        continue

with open("stat_counts_tmax.txt", "a") as f:
    json.dump(counts, f, indent=2)